In [1]:
#全数据
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat,get_feature_names

#数据加载
ratings_data = pd.read_csv("ratings.dat", delimiter='\:\:',header=None)
ratings_data.columns = ['UserID','MovieID','Rating','Timestamp']
print(ratings_data)
movies_data = pd.read_csv("movies.dat", delimiter='\:\:',header=None)
movies_data.columns = ['MovieID','Title','Genres']
users_data = pd.read_csv("users.dat", delimiter='\:\:',header=None)
users_data.columns = ['UserID','Gender','Age','Occupation','Zip-code']
# data = pd.read_csv("movielens_sample.txt")
data1 =  pd.merge(ratings_data, movies_data, on="MovieID", how="inner")
print(data1)
data =  pd.merge(data1, users_data, on="UserID", how="inner")
#data
print(data)
sparse_features = ["UserID", "MovieID", "Gender", "Age", "Occupation", "Zip-code"]
target = ['Rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
print(fixlen_feature_columns)
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

<ipython-input-1-f9b45cd208d3>:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings_data = pd.read_csv("ratings.dat", delimiter='\:\:',header=None)


         UserID  MovieID  Rating  Timestamp
0             1     1193       5  978300760
1             1      661       3  978302109
2             1      914       3  978301968
3             1     3408       4  978300275
4             1     2355       5  978824291
...         ...      ...     ...        ...
1000204    6040     1091       1  956716541
1000205    6040     1094       5  956704887
1000206    6040      562       5  956704746
1000207    6040     1096       4  956715648
1000208    6040     1097       4  956715569

[1000209 rows x 4 columns]


<ipython-input-1-f9b45cd208d3>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_data = pd.read_csv("movies.dat", delimiter='\:\:',header=None)
<ipython-input-1-f9b45cd208d3>:15: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users_data = pd.read_csv("users.dat", delimiter='\:\:',header=None)


         UserID  MovieID  Rating  Timestamp  \
0             1     1193       5  978300760   
1             2     1193       5  978298413   
2            12     1193       4  978220179   
3            15     1193       4  978199279   
4            17     1193       5  978158471   
...         ...      ...     ...        ...   
1000204    5949     2198       5  958846401   
1000205    5675     2703       3  976029116   
1000206    5780     2845       1  958153068   
1000207    5851     3607       5  957756608   
1000208    5938     2909       4  957273353   

                                               Title                Genres  
0             One Flew Over the Cuckoo's Nest (1975)                 Drama  
1             One Flew Over the Cuckoo's Nest (1975)                 Drama  
2             One Flew Over the Cuckoo's Nest (1975)                 Drama  
3             One Flew Over the Cuckoo's Nest (1975)                 Drama  
4             One Flew Over the Cuckoo's Nest (197

e:\Users\huangbo\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


2501/2501 [==============================] - 8s 3ms/step - loss: 0.9979 - mse: 0.9972 - val_loss: 0.8290 - val_mse: 0.8278
test RMSE 0.905317623820502
